In [6]:
import os
import json
import traceback
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
import pandas as pd
import PyPDF2

In [7]:
load_dotenv()

True

In [8]:
key=os.getenv("OPEN_API_KEY")

In [9]:
print(key)

sk-6HEnv0vJe0Iju5edIeTflWmW0YH7QMf2NoJ2f1zRq-T3BlbkFJ7YfS5IMl-023K6PyzlKwKsYKmHqCn4nKhgXH_Q6kAA


In [10]:
llm=ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo",temperature=0.7)

d:\GenAI_Beginner\DAY 6- Automated-MCQ-Generator Project\env\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C0ADB2B910>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C0ADB38AC0>, openai_api_key='sk-6HEnv0vJe0Iju5edIeTflWmW0YH7QMf2NoJ2f1zRq-T3BlbkFJ7YfS5IMl-023K6PyzlKwKsYKmHqCn4nKhgXH_Q6kAA', openai_proxy='')

In [12]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [13]:
TEMPLATE = """
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [14]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [15]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

d:\GenAI_Beginner\DAY 6- Automated-MCQ-Generator Project\env\lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [16]:
TEMPLATE2 = """
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [20]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"], template=TEMPLATE2
)

In [18]:
review_chain = LLMChain(
    llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True
)

In [19]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True,
)

In [21]:
PATH = "D:\GenAI_Beginner\DAY 6- Automated-MCQ-Generator Project\data.txt"

In [22]:
print(PATH)

D:\GenAI_Beginner\DAY 6- Automated-MCQ-Generator Project\data.txt


In [24]:
with open(PATH,"r") as file:
    TEXT=file.read()

In [ ]:
print(TEXT)